In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import math
import os
import sys
import warnings
import argparse
import numpy as np
import yaml
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset,Sampler
from torchvision import datasets,transforms
import torchvision.transforms as transforms
from torch.nn import functional as F

sys.path.append("/home/shiyi/gpu/gpu/")
sys.path.append("/home/shiyi/colposcopy/colposcopy/util")
img_dir = "/data/colposcopy/colposcopy"
img_dir_1 = "/data/colposcopy/colposcopy_1/"

from gpu_allocation import set_gpu

import csv

error = glob(os.path.join(img_dir,"train*","*","2845.jpg"))+ \
    glob(os.path.join(img_dir,"train*","*","5893.jpg"))+glob(os.path.join(img_dir,"train*","*","5892.jpg"))
print(error)

from gpu_allocation import set_gpu
corrupt_set  = set()
types = [1,2,3]

type_1_train = [i for i in glob(os.path.join(img_dir_1,"train*","Type_1","*.jpg")) if i not in error]
type_2_train = [i for i in glob(os.path.join(img_dir_1,"train*","Type_2","*.jpg")) if i not in error]
type_3_train = [i for i in glob(os.path.join(img_dir_1,"train*","Type_3","*.jpg")) if i not in error]

img_test   = glob(os.path.join(img_dir_1,"test","*.jpg"))
img_test_wo_answer = glob(os.path.join(img_dir_1,"test_stg2","*.jpg"))

img_paths = type_1_train+type_2_train+type_3_train+img_test+img_test_wo_answer
print(len(img_paths))

[u'/data/colposcopy/colposcopy/train_stg2/Type_2/2845.jpg', u'/data/colposcopy/colposcopy/train_stg2/Type_1/5893.jpg', u'/data/colposcopy/colposcopy/train_stg2/Type_2/5892.jpg']
12153


In [4]:
class ColpoProcessDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self, index):
        path = img_paths[index]
        x = cv2.imread(path)
        if x is None:
            return  None,None
        return x
    def __len__(self):
        return len(img_paths)

def data_label_alter(csv_path):
    def data_label_del(target_img_path):
        for i in img_paths:
            if i.split('/')[-1]==target_img_path:
                img_paths.remove(i)
                #print ("%s removed"%i)
                return
                
    def data_label_exchange(target_img_path,src,target):
        for i in range(len(img_paths)):
            path_token = img_paths[i].split('/')
            if path_token[-1]==target_img_path and src == path_token[-2]:
                #print (target_img_path,src,target)
                #print (path_token[-2],src,target)
                path_token[-2] = target
               #print ("From",img_paths[i])
                img_paths[i] = '/'.join(path_token)
               # print ("To",img_paths[i])
                return 
                
        
    with open(csv_path) as f:
        f_csv = csv.reader(f)
        index = 0
        for row in f_csv:
            if index == 0:
                index += 1
                continue
            if row[-1] == 'remove':
                data_label_del(row[0])
            else:
                data_label_exchange(row[0],row[1],row[2])

            
def cvt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def preprocess(img): 
    r_c = img[:,:,2]
    g_c = img[:,:,1]
    b_c = img[:,:,0]
    _,r_thres = cv2.threshold(r_c, 210, 255, cv2.THRESH_BINARY)
    _,g_thres = cv2.threshold(g_c, 208, 255, cv2.THRESH_BINARY)
    _,b_thres = cv2.threshold(b_c, 206, 255, cv2.THRESH_BINARY)
    mixed = cv2.UMat(r_thres & b_thres & g_thres)
    
    kernel_dilate = np.ones((18, 18), np.uint8)
    mask = cv2.dilate(mixed,kernel_dilate)
    dst = cv2.inpaint(img,mask,14,cv2.INPAINT_TELEA)
    return cvt(dst)

def collate_fn1(batch):
    x = [item[0] for item in batch if item[0] is not None]
    y = [item[1] for item in batch if item[0] is not None]
    return x,y

In [5]:
print(img_paths[0].split('/')[4:])
data_label_alter(os.path.join(img_dir,"fixed_labels_v2.csv"))
data_label_alter(os.path.join(img_dir,"removed_files.csv"))

[u'train', u'Type_1', u'732.jpg']


In [6]:
print(len(img_paths))

12121


In [20]:
import threading
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
num_gpu = 2
set_gpu(num_gpu)

class myThread (threading.Thread):   
    def __init__(self, threadID, img_num_per_thread):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.img_num_per_thread = img_num_per_thread
        
    def run(self):     
        start = self.img_num_per_thread*self.threadID
        end = min(start+self.img_num_per_thread,len(img_paths))
        index = 0
        for img_path in img_paths[start:end]:
            img_path_target = os.path.join(img_dir_1 + '/'.join(img_path.split('/')[4:]))
            index += 1
            img = cv2.imread(img_path)
            if img is None:
                continue
            img_para = preprocess(img)
            print("Thread {}, number {}".format(self.threadID,index))
            cv2.imwrite(img_path_target,img_para,[int(cv2.IMWRITE_JPEG_QUALITY),100])

num_thread = num_gpu
img_num_per_thread = len(img_paths)//num_thread 
thread_res = list()
for i in range(num_thread):
    thread_res.append(myThread(i,img_num_per_thread))

for i in range(num_thread): 
    thread_res[i].start()


('Found 4 GPU(s)', 'only 2 gpu below threshold')
Using GPU 1,0
Thread 0, number 1
Thread 0, number 2
Thread 0, number 3
Thread 0, number 4
Thread 0, number 5
Thread 0, number 10
Thread 0, number 11
Thread 0, number 12
Thread 0, number 13
Thread 0, number 246
Thread 0, number 14
Thread 0, number 15
Thread 0, number 247
Thread 0, number 16
Thread 0, number 17
Thread 0, number 18
Thread 0, number 249
Thread 0, number 250
Thread 0, number 19
Thread 0, number 22
Thread 0, number 23
Thread 0, number 251Thread 0, number 24

Thread 0, number 25
Thread 0, number 252
Thread 0, number 27
Thread 0, number 28
Thread 0, number 30
Thread 0, number 31
Thread 0, number 254
Thread 0, number 32
Thread 0, number 33
Thread 1, number 1Thread 0, number 255

Thread 1, number 2
Thread 1, number 3
Thread 0, number 256
Thread 0, number 34
Thread 1, number 4
Thread 0, number 35
Thread 0, number 36
Thread 0, number 37
Thread 1, number 5
Thread 0, number 258
Thread 1, number 6
Thread 0, number 38
Thread 1, number 7

Thread 0, number 325
Thread 1, number 80
Thread 1, number 81
Thread 0, number 315
Thread 0, number 316
Thread 0, number 317
Thread 0, number 318
Thread 1, number 84
Thread 0, number 319
Thread 0, number 320
Thread 0, number 321
Thread 0, number 322
Thread 0, number 323
Thread 0, number 324
Thread 0, number 325
Thread 0, number 326
Thread 0, number 327
Thread 0, number 326
Thread 0, number 327
Thread 0, number 328
Thread 0, number 329
Thread 1, number 85
Thread 1, number 86
Thread 1, number 87
Thread 1, number 88
Thread 0, number 328
Thread 0, number 329
Thread 1, number 89
Thread 1, number 90
Thread 1, number 91
Thread 1, number 93
Thread 1, number 94
Thread 1, number 95
Thread 1, number 96
Thread 1, number 98
Thread 1, number 99
Thread 1, number 100
Thread 1, number 101
Thread 1, number 102
Thread 1, number 103
Thread 0, number 330
Thread 0, number 331
Thread 0, number 332
Thread 0, number 333
Thread 0, number 334
Thread 0, number 335
Thread 0, number 336
Thread 0, number 337
Thread 0

Thread 0, number 473
Thread 0, number 473
Thread 0, number 474
Thread 0, number 474
Thread 0, number 475
Thread 0, number 475
Thread 1, number 171
Thread 0, number 476
Thread 0, number 476
Thread 1, number 172
Thread 1, number 173
Thread 0, number 477
Thread 0, number 477
Thread 1, number 174
Thread 1, number 175
Thread 1, number 176
Thread 0, number 478
Thread 0, number 478
Thread 1, number 177
Thread 0, number 479
Thread 1, number 178
Thread 0, number 479
Thread 0, number 480
Thread 0, number 480
Thread 0, number 481
Thread 0, number 481
Thread 0, number 482
Thread 0, number 482
Thread 1, number 179
Thread 0, number 483
Thread 0, number 484
Thread 0, number 483
Thread 0, number 484
Thread 0, number 485
Thread 0, number 486
Thread 0, number 485
Thread 0, number 486
Thread 0, number 487
Thread 0, number 488
Thread 0, number 487
Thread 0, number 488
Thread 0, number 489
Thread 0, number 489
Thread 0, number 490
Thread 0, number 490
Thread 0, number 491
Thread 0, number 491
Thread 1, num

Thread 1, number 262
Thread 0, number 623
Thread 0, number 623
Thread 0, number 624
Thread 0, number 625
Thread 0, number 624
Thread 0, number 625
Thread 0, number 626
Thread 0, number 626
Thread 0, number 627
Thread 0, number 627
Thread 0, number 628
Thread 0, number 628
Thread 0, number 629
Thread 0, number 629
Thread 0, number 630
Thread 0, number 630
Thread 0, number 631
Thread 0, number 631
Thread 0, number 632
Thread 1, number 263Thread 0, number 632

Thread 1, number 264
Thread 1, number 265
Thread 0, number 633
Thread 0, number 633
Thread 0, number 634
Thread 0, number 634
Thread 0, number 635
Thread 0, number 635
Thread 0, number 636
Thread 0, number 636
Thread 0, number 637
Thread 1, number 266
Thread 0, number 637
Thread 1, number 267
Thread 1, number 268
Thread 1, number 269
Thread 0, number 638
Thread 0, number 638
Thread 0, number 639
Thread 0, number 639
Thread 0, number 640
Thread 0, number 640
Thread 0, number 641
Thread 0, number 642
Thread 0, number 641
Thread 0, num

Thread 0, number 766
Thread 0, number 767
Thread 0, number 767
Thread 0, number 768
Thread 0, number 769
Thread 0, number 768
Thread 0, number 769
Thread 0, number 770
Thread 0, number 770
Thread 0, number 771
Thread 0, number 771
Thread 0, number 772
Thread 0, number 772
Thread 0, number 773
Thread 0, number 773
Thread 0, number 774
Thread 0, number 774
Thread 1, number 366
Thread 0, number 775
Thread 0, number 775
Thread 1, number 367
Thread 1, number 368
Thread 0, number 776
Thread 0, number 776
Thread 0, number 777
Thread 0, number 777
Thread 0, number 778
Thread 0, number 778
Thread 0, number 779
Thread 0, number 779
Thread 0, number 780
Thread 0, number 780
Thread 0, number 781
Thread 0, number 781
Thread 0, number 782
Thread 0, number 782
Thread 1, number 369
Thread 1, number 370
Thread 1, number 371
Thread 1, number 372
Thread 0, number 783
Thread 1, number 373
Thread 1, number 374Thread 0, number 783

Thread 0, number 784
Thread 0, number 784
Thread 0, number 785
Thread 0, num

...

Thread 1, number 3112
Thread 1, number 3113
Thread 1, number 3114
Thread 1, number 3115
Thread 1, number 3116
Thread 1, number 3117
Thread 1, number 3118
Thread 1, number 3119
Thread 0, number 3554
Thread 1, number 3120
Thread 0, number 3554
Thread 1, number 3121
Thread 1, number 3122
Thread 0, number 3555
Thread 0, number 3555
Thread 0, number 3556
Thread 0, number 3556
Thread 0, number 3557
Thread 0, number 3557
Thread 0, number 3558
Thread 0, number 3558
Thread 0, number 3559
Thread 0, number 3559
Thread 0, number 3560Thread 0, number 3560

Thread 0, number 3561
Thread 0, number 3561
Thread 1, number 3123
Thread 1, number 3124
Thread 0, number 3562
Thread 0, number 3562
Thread 1, number 3125Thread 0, number 3563

Thread 0, number 3563
Thread 0, number 3564
Thread 0, number 3564
Thread 0, number 3565
Thread 0, number 3565
Thread 0, number 3566
Thread 0, number 3566
Thread 0, number 3567
Thread 0, number 3567
Thread 1, number 3126
Thread 0, number 3568
Thread 0, number 3568
Thread 0, 

Thread 1, number 3317
Thread 1, number 3318
Thread 1, number 3319
Thread 0, number 3638
Thread 0, number 3638
Thread 0, number 3639
Thread 0, number 3640
Thread 0, number 3639
Thread 0, number 3640
Thread 1, number 3320
Thread 1, number 3321
Thread 1, number 3322
Thread 1, number 3323
Thread 1, number 3324
Thread 1, number 3325
Thread 1, number 3326
Thread 1, number 3327
Thread 1, number 3328
Thread 1, number 3329
Thread 1, number 3330
Thread 1, number 3331
Thread 1, number 3332
Thread 1, number 3333
Thread 1, number 3334
Thread 1, number 3335
Thread 1, number 3336
Thread 1, number 3337
Thread 1, number 3338
Thread 1, number 3339
Thread 0, number 3641Thread 0, number 3641

Thread 1, number 3340
Thread 1, number 3341
Thread 1, number 3342
Thread 1, number 3343
Thread 1, number 3344
Thread 1, number 3345
Thread 1, number 3346
Thread 1, number 3347
Thread 1, number 3348
Thread 1, number 3349
Thread 1, number 3350
Thread 1, number 3351
Thread 1, number 3352
Thread 1, number 3353
Thread 1, 

Thread 0, number 3725
Thread 0, number 3726
Thread 0, number 3725
Thread 0, number 3726
Thread 1, number 3516
Thread 1, number 3517
Thread 1, number 3518
Thread 1, number 3519
Thread 1, number 3520
Thread 0, number 3727
Thread 1, number 3521
Thread 1, number 3522
Thread 1, number 3523
Thread 0, number 3727
Thread 0, number 3728
Thread 1, number 3524
Thread 1, number 3525
Thread 1, number 3526
Thread 0, number 3728
Thread 1, number 3527
Thread 1, number 3528
Thread 1, number 3529
Thread 1, number 3530
Thread 1, number 3531
Thread 1, number 3532
Thread 1, number 3533
Thread 1, number 3534
Thread 1, number 3535
Thread 1, number 3536
Thread 1, number 3537
Thread 1, number 3538
Thread 1, number 3539
Thread 1, number 3540
Thread 1, number 3541
Thread 1, number 3542
Thread 1, number 3543
Thread 1, number 3544
Thread 0, number 3729
Thread 0, number 3729
Thread 1, number 3545
Thread 1, number 3546
Thread 1, number 3547
Thread 1, number 3548
Thread 0, number 3730Thread 1, number 3549

Thread 0, 

Thread 0, number 3816
Thread 0, number 3816
Thread 0, number 3817
Thread 0, number 3817
Thread 1, number 3707
Thread 0, number 3818
Thread 0, number 3818
Thread 0, number 3819
Thread 1, number 3708
Thread 1, number 3709
Thread 1, number 3710
Thread 1, number 3711
Thread 1, number 3712
Thread 1, number 3713
Thread 1, number 3714
Thread 0, number 3820
Thread 0, number 3820
Thread 1, number 3715
Thread 1, number 3716
Thread 1, number 3717
Thread 1, number 3718
Thread 0, number 3821
Thread 0, number 3822
Thread 0, number 3821
Thread 0, number 3822
Thread 0, number 3823
Thread 0, number 3823
Thread 1, number 3719
Thread 1, number 3720
Thread 1, number 3721
Thread 0, number 3824
Thread 0, number 3824
Thread 0, number 3825
Thread 0, number 3825
Thread 0, number 3826
Thread 0, number 3826
Thread 1, number 3722
Thread 0, number 3827
Thread 0, number 3827
Thread 0, number 3828
Thread 0, number 3828
Thread 1, number 3723
Thread 0, number 3829
Thread 0, number 3829
Thread 0, number 3830
Thread 1, 

Thread 1, number 3879
Thread 1, number 3880
Thread 0, number 3917
Thread 0, number 3917
Thread 1, number 3881
Thread 0, number 3918
Thread 0, number 3918
Thread 1, number 3882
Thread 1, number 3883
Thread 1, number 3884
Thread 1, number 3885
Thread 0, number 3919
Thread 0, number 3919
Thread 0, number 3920
Thread 0, number 3920
Thread 0, number 3921
Thread 0, number 3921
Thread 1, number 3886
Thread 0, number 3922
Thread 0, number 3922
Thread 1, number 3887
Thread 1, number 3888
Thread 0, number 3923
Thread 0, number 3923
Thread 0, number 3924
Thread 0, number 3924
Thread 1, number 3889


In [ ]:
import csv
csv_path = (img_dir +'/'+ "removed_files.csv")
f = open(csv_path)
f_csv = csv.reader(f)
for row in f_csv:
    print(row)

In [ ]:
def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else: 
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i-position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif(height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()    
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area =  maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea
            

def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

def cropCircle(img):
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    img = cv2.resize(img, dsize=tile_size)
            
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
            
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    
    rect = maxRect(ff)
    rectangle = [min(rect[0],rect[2]), max(rect[0],rect[2]), min(rect[1],rect[3]), max(rect[1],rect[3])]
    img_crop = img[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    
    return [img_crop, rectangle, tile_size]

def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

def get_and_crop_image(image_id, image_type):
    img = get_image_data(image_id, image_type)
    initial_shape = img.shape
    [img, rectangle_cropCircle, tile_size] = cropCircle(img)
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = Ra_space(imgLab, 1.0, 150)
    a_channel = np.reshape(Ra[:,1], (w,h))
    
    g = mixture.GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
    image_array_sample = shuffle(Ra, random_state=0)[:1000]
    g.fit(image_array_sample)
    labels = g.predict(Ra)
    labels += 1 # Add 1 to avoid labeling as 0 since regionprops ignores the 0-label.
    
    # The cluster that has the highest a-mean is selected.
    labels_2D = np.reshape(labels, (w,h))
    gg_labels_regions = measure.regionprops(labels_2D, intensity_image = a_channel)
    gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
    cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1

    mask = np.zeros((w * h,1),'uint8')
    mask[labels==cervix_cluster] = 255
    mask_2D = np.reshape(mask, (w,h))

    cc_labels = measure.label(mask_2D, background=0)
    regions = measure.regionprops(cc_labels)
    areas = [prop.area for prop in regions]

    regions_label = [prop.label for prop in regions]
    largestCC_label = regions_label[areas.index(max(areas))]
    mask_largestCC = np.zeros((w,h),'uint8')
    mask_largestCC[cc_labels==largestCC_label] = 255

    img_masked = img.copy()
    img_masked[mask_largestCC==0] = (0,0,0)
    img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY);
            
    _,thresh_mask = cv2.threshold(img_masked_gray,0,255,0)
            
    kernel = np.ones((11,11), np.uint8)
    thresh_mask = cv2.dilate(thresh_mask, kernel, iterations = 1)
    thresh_mask = cv2.erode(thresh_mask, kernel, iterations = 2)
    _, contours_mask, _ = cv2.findContours(thresh_mask.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours_mask, key = cv2.contourArea, reverse = True)[0]
    cv2.drawContours(img, main_contour, -1, 255, 3)
    
    x,y,w,h = cv2.boundingRect(main_contour)
    
    rectangle = [x+rectangle_cropCircle[2],
                 y+rectangle_cropCircle[0],
                 w,
                 h,
                 initial_shape[0],
                 initial_shape[1],
                 tile_size[0],
                 tile_size[1]]

    return [image_id, img, rectangle]

In [9]:
print(3//2)

1


In [36]:
def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

def maxHist(lst):
    s = []
    i = 0
    n = len(lst)
    max_val = 0
    while(i<n):
        print (s)
        if len(s)==0 or lst[i]>=lst[s[-1]]:
            s.append(i)
            i += 1
        else:
            top = s.pop()
            val = lst[top] * (i if len(s)<=0 else i-s[-1]-1)
            max_val = max(max_val,val)
    
    while(len(s)>0):
        top = s.pop()
        val = lst[top] * (i if len(s)<=0 else i-s[-1]-1)
        max_val = max(max_val,val)
    return max_val


[]
0 appended
[0]
1 [] val 6 maxval 6
[]
1 appended
[1]
2 [] val 4 maxval 6
[]
2 appended
[2]
3 appended
[2, 3]
4 [2] val 4 maxval 6
[2]
4 appended
[2, 4]
5 [2] val 6 maxval 6
[2]
5 appended
[2, 5]
6 appended
stack [2, 5, 6]
hist [6, 2, 1, 4, 3, 2, 8]
8
Thread 0, number 3981
Thread 1, number 3971
Thread 1, number 3972
Thread 0, number 3981
Thread 1, number 3973
Thread 1, number 3974
Thread 1, number 3975
Thread 1, number 3976
Thread 0, number 3982
Thread 0, number 3983
Thread 0, number 3982
Thread 0, number 3983
Thread 1, number 3977
Thread 0, number 3984
Thread 1, number 3978
Thread 0, number 3985
Thread 0, number 3984
Thread 0, number 3985
Thread 1, number 3979
Thread 1, number 3980
Thread 1, number 3981
Thread 1, number 3982
Thread 1, number 3983
Thread 1, number 3984
Thread 1, number 3985
Thread 0, number 3986
Thread 0, number 3986
Thread 0, number 3987
Thread 0, number 3987
Thread 0, number 3988
Thread 0, number 3989
Thread 0, number 3988
Thread 0, number 3989
Thread 0, number 399

Thread 1, number 4128
Thread 1, number 4129
Thread 0, number 4083
Thread 0, number 4083
Thread 1, number 4130
Thread 1, number 4131
Thread 1, number 4132
Thread 1, number 4133
Thread 1, number 4134
Thread 1, number 4135
Thread 1, number 4136
Thread 1, number 4137
Thread 0, number 4084
Thread 0, number 4084
Thread 0, number 4085
Thread 0, number 4085
Thread 1, number 4138
Thread 0, number 4086
Thread 1, number 4139
Thread 0, number 4086
Thread 1, number 4140
Thread 1, number 4141
Thread 0, number 4087
Thread 0, number 4087
Thread 0, number 4088
Thread 0, number 4088
Thread 1, number 4142
Thread 1, number 4143
Thread 1, number 4144
Thread 1, number 4145
Thread 1, number 4146
Thread 0, number 4089
Thread 0, number 4089
Thread 0, number 4090
Thread 1, number 4147
Thread 0, number 4091
Thread 1, number 4148
Thread 0, number 4090
Thread 0, number 4091
Thread 0, number 4092
Thread 0, number 4092
Thread 0, number 4093
Thread 0, number 4093
Thread 0, number 4094
Thread 0, number 4094
Thread 1, 